## Package

### General

In [3]:
import re
import numpy as np
import pandas as pd
import mne
import time
import multiprocessing as mp
from joblib import Parallel, delayed
from itertools import combinations, permutations
import csv
import os
import math
from itertools import chain
from tqdm import tqdm
import gc
%matplotlib inline

C:\Users\vita7\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\vita7\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\vita7\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


### Statistics and Muchine Learning

In [2]:
from scipy import stats
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
from sklearn import feature_selection
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix, cohen_kappa_score
from sklearn.model_selection import learning_curve
from sklearn.model_selection import validation_curve
from sklearn.model_selection import GridSearchCV
from bayes_opt import BayesianOptimization
from sklearn.model_selection import cross_val_score
import pickle
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model  import LogisticRegression
from sklearn.naive_bayes import GaussianNB

### Signal processing

In [5]:
import scipy.signal as signal
from pyod.models.abod import ABOD

C:\Users\vita7\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


### Feature

In [6]:
from entropy import *
from mne_features import *

C:\Users\vita7\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


### Visualization

In [7]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

C:\Users\vita7\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


### Special processing

In [8]:
from chaos_master import ssa_core

## Parameter

In [9]:
# 資料庫 EDF檔路徑
recording_path = "C:/Users/vita7/OneDrive/Documents/Github/EEG_SleepStage_Analysis/data/SC/recordings/"

# 批量開啟檔案的正則表達式
record_regu = r"SC\d*\D\d\-PSG.edf"
annot_regu = r"SC\d*\D\D\-Hypnogram.edf"

# EDF檔通道類型-
# eeg - 要選用的EEG通道
# misc - 忽略的通道
channel_types = {
    'EEG Fpz-Cz': 'eeg',
    'EEG Pz-Oz': 'misc',
    'EOG horizontal': 'misc',
    'Resp oro-nasal': 'misc',
    'EMG submental': 'misc',
    'Temp rectal': 'misc',
    'Event marker': 'misc'
}

# 睡眠階段編碼-
# Annotation-EDF檔中的睡眠階段為 W 1 2 3 4 R。
# 我們將 3 4 視為一個睡眠階段,因此統一編號為 4。
sleep_stage = {
    'Sleep stage W': 1,
    'Sleep stage 1': 2,
    'Sleep stage 2': 3,
    'Sleep stage 3': 4,
    'Sleep stage 4': 4,
    'Sleep stage R': 5
}

# 事件ID
# 對應於編碼後的事件名稱
event_id = {
    'Sleep stage W': 1,
    'Sleep stage 1': 2,
    'Sleep stage 2': 3,
    'Sleep stage 3/4': 4,
    'Sleep stage R': 5,
}

# 計算函數名稱與函數個數
funcs_n = [
    'Variance', 'Peak to Peak Amplitude', 'Zero Crossings', 'Average Power',
    'Spectral slope', 'Mean Frequency', 'Hjorth complexity',
    'Spectral Entropy', 'Permutation Entropy', 'SVD Entropy', 'Sample Entropy',
    'First-order AutoCorrelation Coefficient'
]
funcs = 12

# 包含原始信號在內的六個頻帶名稱
six_bands_n = [
    'EEG Fpz-Cz', 'EEG Fpz-Cz Delta', 'EEG Fpz-Cz Theta', 'EEG Fpz-Cz Alpha',
    'EEG Fpz-Cz Sigma', 'EEG Fpz-Cz Beta'
]

# 比帶名稱
ratio_n = [
    'EEG Fpz-Cz DOR', 'EEG Fpz-Cz TOR', 'EEG Fpz-Cz AOR', 'EEG Fpz-Cz SOR',
    'EEG Fpz-Cz BOR', 'EEG Fpz-Cz TDR', 'EEG Fpz-Cz ADR', 'EEG Fpz-Cz SDR',
    'EEG Fpz-Cz BDR', 'EEG Fpz-Cz ATR', 'EEG Fpz-Cz STR', 'EEG Fpz-Cz BTR',
    'EEG Fpz-Cz SAR', 'EEG Fpz-Cz BAR', 'EEG Fpz-Cz BSR'
]

# 頻帶名稱
frbands_n = six_bands_n + ratio_n

# Epoch持續持間 (s)
ep_duration = 10.

# 平滑因子:
#我們將10s數據量視為1 epoch,但實際上 R&K rule規範1 epoch 為30s數據量,因此平滑因子為3倍。
smoothing_factor = 3

# 採樣頻率
sfreq = 100.

# 分析的帶通上下截止頻率
# 下截止頻率為 0.5Hz 可以濾除 motion artifact。
# 上截止頻率為 30Hz 可以濾除市電 50(60)Hz 干擾,並且EEG訊號涵蓋頻率大多在30Hz以下。
l_freq = 0.5
h_freq = 30

# 通道數
chs = 1

# Number of recorduings
rec_n = 39

# testing dataframe index
tei = [12,14,19,23,27,28,30,37]
# training dataframe index
tri = set(range(rec_n)) - set(tei)